In [ ]:
from dotenv import load_dotenv
from pathlib import Path
import os
import sys
import openai
import csv

env_path = Path('../..') / '.env'
load_dotenv(dotenv_path=env_path)

print(os.getcwd())
sys.path.append('/Users/davidwei/Documents/SellScale/sellscale-api/notebooks/gpt_api')

OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
openai.api_key = OPENAI_API_KEY

CURRENT_OPENAI_CHAT_GPT_MODEL = "gpt-3.5-turbo"

In [ ]:
import time

def chat_completion(prompt: str):
    response = openai.ChatCompletion.create(
    model=CURRENT_OPENAI_CHAT_GPT_MODEL,
    messages=[{
                "role": "user", "content": prompt
            }]
    )
    if response is None or response["choices"] is None or len(response["choices"]) == 0:
        return ""

    choices = response["choices"]
    top_choice = choices[0]
    preview = top_choice["message"]["content"].strip()

    print(time.time())

    return preview

In [ ]:
import queue
import time
import concurrent.futures

def make_api_call(results_queue):
    response = chat_completion("Tell me a joke.")
    # Process the response as needed
    results_queue.put(response)  # Store the response in the queue

def parallel_api_calls():
    start = time.time()
    results_queue = queue.Queue()  # Queue to store the results

    max_threads = 3

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = [executor.submit(make_api_call, results_queue) for i in range(20)]
        # Wait for all tasks to complete
        concurrent.futures.wait(futures)

    # Retrieve and process the results
    while not results_queue.empty():
        result = results_queue.get()
        print(f"Response: {result}")

    end = time.time()
    print(f"Time taken: {end - start} seconds")

# Call the function
parallel_api_calls()


In [ ]:
import time


def sequential_api_calls():
    start = time.time()
    for i in range(20):
        response = chat_completion("Tell me a joke.")
        print(f"Response: {response}")

    end = time.time()
    print(f"Time taken: {end - start} seconds")

sequential_api_calls()
